In [0]:
# Fontes e bibliotecas
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import *
import urllib.request # Requisição para baixar arquivo na internet
import gzip # descompactar gzip
import os # utilizar sistema operacional

# Parâmetros
# Tinha um erro no link do arquivo: estava como 'sour-ce' e na verdade é 'source'
url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz'
local_path = "/tmp/order.json.gz"
bronze_path = "dbfs:/mnt/bronze/raw_order" 

# Baixar arquivo
urllib.request.urlretrieve(url, local_path)

# Leitura do JSON
# file:// é utilizado para não buscar DBFS
# multline porque o json tem mais de uma linha por objeto
df = spark.read.json(f"file://{local_path}", multiLine=True)

# Verificando
df.columns()        
df.schema.simpleString()  
df.display()

# Escrevendo no DBFS
df.write.format("delta").mode("overwrite").save(bronze_path)

# Registro como tabela para manipulação SQL
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS raw_order
          USING DELTA
          LOCATION '{bronze_path}'
          """)